In [1]:
# Import libraries for data wrangling, preprocessing and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

C:\Users\JLipemelton\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("Titanic Data/train.csv")
seed = 5
np.random.seed(seed)

In [3]:
data.head()
#data.info()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# data.drop("Name", axis=1, inplace=True)
# data.drop("Ticket", axis=1, inplace=True)
# data.head()
# count = data.isna().sum()
avg = data['Age'].mean()
data['Age'].fillna(avg, inplace=True)
data['Embarked'].fillna(2, inplace=True)
# final_count = data.isna().sum()
# print("\nCOUNT:\n"+ str(count) + "\nFINAL COUNT\n" + str(final_count) +"\n")

In [5]:
print(data.head())
data['Embarked'].value_counts()
cleanup_nums = { 'Embarked': {'S':1, 'C':2, 'Q':3} }
data.replace(cleanup_nums, inplace=True)
print(data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

In [6]:
prediction_var = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
x = data[prediction_var].values
y = data.Survived.values
length = x.shape[0]
split = .9
point = int(split*length)

x_train = x[:point]
y_train = y[:point]
x_test = x[point:]
y_test = y[point:]
inputCount = len(prediction_var)

In [7]:
model = Sequential()
model.add(Dense(100, input_dim=inputCount, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.6))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
#print(x[0])
#model.fit(x=X, y=y, epochs=10, steps_per_epoch=100, validation_split=0.1, shuffle=True, verbose=1, validation_steps=100)
model.fit(x_train, y_train, epochs=50, batch_size=128, verbose=1)
print("\n\n")
score=model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/50
801/801 [==============================] - 0s 41us/step - loss: 0.6231 - acc: 0.6554
Epoch 2/50
801/801 [==============================] - 0s 50us/step - loss: 0.6398 - acc: 0.6679
Epoch 3/50
801/801 [==============================] - 0s 52us/step - loss: 0.6414 - acc: 0.6467
Epoch 4/50
801/801 [==============================] - 0s 48us/step - loss: 0.6274 - acc: 0.6804
Epoch 5/50
801/801 [==============================] - 0s 51us/step - loss: 0.6303 - acc: 0.6567
Epoch 6/50
801/801 [==============================] - 0s 50us/step - loss: 0.6263 - acc: 0.6592
Epoch 7/50
801/801 [==============================] - 0s 61us/step - loss: 0.6346 - acc: 0.6617
Epoch 8/50
801/801 [==============================] - 0s 58us/step - loss: 0.6347 - acc: 0.6692
Epoch 9/50
801/801 [==============================] - 0s 44us/step - loss: 0.6265 - acc: 0.6704
Epoch 10/50
801/801 [==============================] - ETA: 0s - loss: 0.5926 - acc: 0.679 - 0s 55us/step - loss: 0.6247 - acc: 0.6492
E

In [14]:
predictions = model.predict(x_test)
predictions = [round(float(i)) for i in predictions]

In [15]:
correct=[]
for i in range(len(predictions)):
    if predictions[i] == y_test[i]:
        correct.append(1)
    else:
        correct.append(0)

In [16]:
sum(correct)/len(correct)

0.7666666666666667

In [17]:
print(data['Survived'].sum())
#data['Survived']
1-342/890

342


0.6157303370786518